# Trích Xuất Thông Tin Hợp Đồng Khách Sạn

Notebook này thực hiện các bước xử lý từng phần để trích xuất thông tin từ file PDF hợp đồng khách sạn và kiểm tra dữ liệu bằng pandas.

In [2]:
# Import các thư viện cần thiết
import os
import re
import json
import pandas as pd
import pdfplumber
from pathlib import Path

# Đường dẫn thư mục input và output
INPUT_DIR = "../store/input"
OUTPUT_DIR = "../store/output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

## 1. Danh sách các file PDF trong thư mục input

In [3]:
# Liệt kê các file PDF trong thư mục input
pdf_files = [f for f in Path(INPUT_DIR).glob("*.pdf")]
print(f"Tìm thấy {len(pdf_files)} file PDF:")
for pdf in pdf_files:
    print(f"- {pdf.name}")

Tìm thấy 4 file PDF:
- 231207_Bliss Hôi An Beach Resort_Contract 2024 - 2025.pdf
- 240822 - Scan - HD Chalcedony 2025.pdf
- HDONG GIA PHONG HIDDEN CHARM - EASIA INBOUND 2025.pdf
- your_contract_file.pdf


## 2. Đọc và trích xuất văn bản từ một file PDF

In [4]:
# Chọn một file PDF để xử lý
sample_pdf = pdf_files[3] if pdf_files else None

if sample_pdf:
    print(f"Đang xử lý file: {sample_pdf.name}")
    print("================================================================")
    # Mở file PDF và trích xuất văn bản
    with pdfplumber.open(sample_pdf) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    #print(text[:1000])  # Hiển thị 1000 ký tự đầu tiên của văn bản
    print(text)
else:
    print("Không tìm thấy file PDF nào trong thư mục input.")

Đang xử lý file: your_contract_file.pdf
HỢP ĐỒNG DỊCH VỤ 2024- 2025
Validity: 01 Nov 2024 – 31 Oct 2025
Market: All markets (except Korean Market)
Thỏa thuận Giá và Dịch vụ này là giữa Bliss Hội An Beach Resort & Wellness và Easia Corp nêu rõ các điều kiện và dịch vụ
cụ thể sẽ được cung cấp như sau:
This Rate and Service Agreement is Bliss Hoi An Beach Resort & Wellness with Easia Corp as follows:
: CÔNG TY TNHH DU LỊCH NGHỈ DƯỠNG LÀNG BIỂN NHIỆT ĐỚI HỘI AN
BÊN A/PARTY A
(Tropical Beach Village Resort Hoian Company Limited)
Địa chỉ/Address : Thôn Tân An, Xã Bình Minh, Huyện Thăng Bình, Tỉnh Quảng Nam, Việt Nam
Điện thoại/Hotline : +84 911 889 558
Mã số thuế/Tax Code : 4 0 0 1 0 1 1 7 9 5
Đại diện/Represented by : Ông Nguyễn Giang Nam
Chức danh/Title : Tổng giám đốc
BÊN B/PARTY B : CÔNG TY CỔ PHẦN MIỀN Á ĐÔNG
Địa chỉ/Address : Khách sạn ATS – Số 33B Phạm Ngũ Lão, P. Phan Chu Trinh, Q. Hoàn Kiếm, TP. Hanoi
Điện thoại/Direct line : 024.39331362
Mã số thuế/Tax code : 01 01 05 11 77
Đại diệ

## 3.1 Trích xuất thông tin từ văn bản bằng regex

In [ ]:
# Hàm trích xuất thông tin từ văn bản
def extract_contract_data(pdf_text):
    data = {}
    data["ten_khach_san"] = re.search(r"Tên Khách Sạn:\s*(.+)", pdf_text, re.IGNORECASE).group(1).strip() if re.search(r"Tên Khách Sạn:\s*(.+)", pdf_text, re.IGNORECASE) else None
    data["dia_chi"] = re.search(r"Địa Chỉ:\s*(.+)", pdf_text, re.IGNORECASE).group(1).strip() if re.search(r"Địa Chỉ:\s*(.+)", pdf_text, re.IGNORECASE) else None
    data["dia_chi"] = re.search(r"Địa Chỉ:\s*(.+)", pdf_text, re.IGNORECASE).group(1).strip() if re.search(r"Địa Chỉ:\s*(.+)", pdf_text, re.IGNORECASE) else None
    data["gia"] = re.search(r"Giá:\s*(.+)", pdf_text, re.IGNORECASE).group(1).strip() if re.search(r"Giá:\s*(.+)", pdf_text, re.IGNORECASE) else None
    # Thêm các regex khác để trích xuất thông tin cần thiết
    return data

# Trích xuất thông tin từ văn bản
if sample_pdf:
    extracted_data = extract_contract_data(text)
    print(json.dumps(extracted_data, ensure_ascii=False, indent=2))

{
  "ten_khach_san": null,
  "dia_chi": null,
  "gia": null
}


## 3.2 Trích xuất thông tin từ văn bản bằng Regex

In [6]:
def extract_with_regex(text):
    info = {}
    patterns = {
        "hotel_name": r"Khách sạn ([A-Z\w\s]+)",
        "address": r"Địa chỉ ([^\n]+)",
        "contract_number": r"Số hợp đồng: ([\w\d/-]+)",
        "start_date": r"(\d{1,2}/\d{1,2}/\d{4})",
        "end_date": r"đến ngày (\d{1,2}/\d{1,2}/\d{4})",
        "total_value": r"Tổng giá trị(?: hợp đồng)?: ([\d.,]+ VNĐ)"
    }
    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            info[key] = match.group(1).strip()
    return info

info_regex = extract_with_regex(text)
print("===> Kết quả từ Regex:")
print(json.dumps(info_regex, indent=2, ensure_ascii=False))
pd.DataFrame([info_regex])

===> Kết quả từ Regex:
{
  "hotel_name": "ATS",
  "address": "ngân hàng/Beneficiary Address : 44 Lac Long Quan Str, Xuan La Precinct, Tay Ho Ward, Ha Noi, Vietnam",
  "start_date": "01/11/2024"
}


,hotel_name,address,start_date
0,ATS,ngân hàng/Beneficiary Address : 44 Lac Long Qu...,01/11/2024


## 3.2 Trích xuất thông tin từ văn bản bằng spaCy

## 4. Lưu kết quả dưới dạng JSON

In [ ]:
# Lưu kết quả vào file JSON
if sample_pdf:
    output_file = Path(OUTPUT_DIR) / sample_pdf.with_suffix(".json").name
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(extracted_data, f, ensure_ascii=False, indent=2)
    print(f"✅ Đã lưu kết quả vào: {output_file}")

## 5. Hiển thị dữ liệu bằng pandas

In [10]:
# Chuyển dữ liệu thành DataFrame để kiểm tra
if sample_pdf:
    df = pd.DataFrame([extracted_data])
    display(df)

,ten_khach_san,dia_chi
0,None,None
